In [1]:
import cv2
import numpy as np
import glob
np.set_printoptions(formatter={'float': '{: 0.6f}'.format})

def draw(img, corners, imgpts):
    for i in range(0, 4):
        j = (i + 1) if i < 3 else 0
        img = cv2.line(img, tuple(imgpts[i].ravel()), tuple(imgpts[j].ravel()), (0,0,255), 3)
        img = cv2.line(img, tuple(imgpts[i].ravel()), tuple(imgpts[4].ravel()), (0,0,255), 3)
    return img

In [2]:
# Load previously saved data
params = np.load('params.npz')
mtx = params['cameraMatrix']
dist = params['distCoeff']

In [3]:
def showImage(img):
    cv2.namedWindow("Show Image", cv2.WINDOW_NORMAL)
    cv2.imshow("Show Image",img)
    cv2.waitKey(2000)
#     cv2.destroyAllWindows()
# showImage(images[0])

In [4]:
# Make a list of calibration images
images = [cv2.imread(file) for file in glob.glob("../images/CameraCalibration/*.bmp")]

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:8, 0:11].T.reshape(-1,2)

axis = np.float32([[-1,-1,0], [-1,1,0], 
                   [1,1,0], [1,-1,0], [0,0,-2]])

In [7]:
def Q2():
    for img in images[:5]:
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (8,11),None)

        if ret == True:
            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

            # Find the rotation and translation vectors.
            _, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist[0])

            # project 3D points to image plane
            imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

            img = draw(img,corners2,imgpts)
            showImage(img)

    cv2.destroyAllWindows()

In [8]:
Q2()